In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install openpyxl

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle as cPickle
import pandas as pd
from keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from keras.layers import LSTM, Dense, TimeDistributed, Bidirectional
import sklearn.metrics
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras import Model
from keras.layers import Dense,Embedding,LSTM,Dropout,Bidirectional
from keras import regularizers
from keras import models
from openpyxl import Workbook

In [ ]:
path_data = "/content/drive/MyDrive/SuccKD_MultiSpecies/Data/"
path_model = "/content/drive/MyDrive/SuccKD_MultiSpecies/Model/"
path_result = "/content/drive/MyDrive/SuccKD_MultiSpecies/Result_cv/"

In [ ]:
file_train = "Train_E.coli_succ.csv"

In [ ]:
# Build an amino acid dictionary
def twoTupleDic1():
    AA_list_sort = ['G','A','V','L','I','M','P','F','W','S','T','N','Q','Y','C','K','R','H','D','E','X']

    AA_dict = {}
    numm = 1
    for i in AA_list_sort:
        AA_dict[i] = numm
        numm += 1
    return AA_dict
def ProSentence(pro, K):
	sentence = ""
	length = len(pro)
	for i in range(length - K + 1):
		sentence += pro[i: i + K] + " "
    #delete extra space
	sentence = sentence[0 : len(sentence) - 1]
	return sentence
k =1#1-gram
word_index1 = twoTupleDic1()
vocab_size = len(word_index1)


In [ ]:
#  Load train data and pre-processing
df_train =pd.read_csv(path_data+file_train, delimiter= ',')

texts_train =[]
for i in df_train['Peptide_WS_33']:
  temp = ProSentence(i,k)
  texts_train.append(temp)
df_train['k_mer'] =texts_train
train_sequences = []
for each in texts_train:
    each_index_list = []
    each = each.split(' ')
    for i in each:
        each_index_list.append(word_index1[i])
    train_sequences.append(each_index_list)
# Tokenizer
data_token = []
for i in df_train['k_mer']:
   data_token.append(i.split())
MAX_SEQUENCE_LENGTH = len(data_token[1])

Xtrain = pad_sequences(train_sequences, maxlen=MAX_SEQUENCE_LENGTH)
ytrain = np.array(df_train['Label'])
ytrain = np.array(ytrain)
#one-hot encoding on the labels
lb = LabelBinarizer()
ytrain= lb.fit_transform(ytrain)
ytrain = to_categorical(ytrain)
ytrain.shape


(5826, 2)

In [ ]:
TIME_STEPS = MAX_SEQUENCE_LENGTH
INPUT_SIZE = 300

In [ ]:
# KD2_succi
import tensorflow as tf
from tensorflow.keras import optimizers
from tensorflow.keras import models, layers, losses

# Teacher 1
def create_cnn_model():
    model = models.Sequential()
    model.add(layers.Embedding(vocab_size+1, 300, input_length=MAX_SEQUENCE_LENGTH,trainable=True))
    model.add(layers.Dropout(0.4))
    model.add(layers.Conv1D(filters=32, kernel_size=3, activation='relu', padding='same'))
    model.add(layers.MaxPooling1D(2))
    model.add(layers.Dropout(0.4))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv1D(filters=32, kernel_size=3, activation='relu'))
    model.add(layers.MaxPooling1D(pool_size=2))
    model.add(layers.Flatten())
    model.add(layers.Dropout(0.4))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(2, activation='softmax'))  # Binary classification (softmax for distillation)
    return model

# Teacher 2
def create_bilstm_model():
    model = models.Sequential()
    model.add(layers.Embedding(vocab_size+1, 300, input_length=MAX_SEQUENCE_LENGTH,trainable=True))
    model.add(layers.Bidirectional(layers.LSTM(units=32, return_sequences=True), input_shape=(TIME_STEPS, INPUT_SIZE)))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.2))
    model.add(layers.Bidirectional(layers.LSTM(units=32, return_sequences=True), input_shape=(TIME_STEPS, INPUT_SIZE)))
    model.add(layers.BatchNormalization())
    model.add(layers.Flatten())
    model.add(layers.Dropout(0.4))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(2, activation='softmax'))  # Binary classification (softmax for distillation)
    return model
# Student
def create_student():
    model = models.Sequential()
    model.add(layers.Embedding(vocab_size+1, 300, input_length=MAX_SEQUENCE_LENGTH,trainable=True))
    model.add(layers.Conv1D(filters=32, kernel_size=3, activation='relu', padding='same'))
    model.add(layers.MaxPooling1D(2))
    model.add(layers.Dropout(0.2))
    model.add(layers.BatchNormalization())
    model.add(layers.Bidirectional(layers.LSTM(units=32, return_sequences=True), input_shape=(TIME_STEPS, INPUT_SIZE)))
    model.add(layers.Flatten())
    model.add(layers.Dropout(0.4))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(2, activation='softmax'))  # Binary classification (softmax for distillation)
    return model
#  KD2_succi: Knowlege distillation
import tensorflow as tf
from tensorflow.keras import optimizers

class Distiller(tf.keras.Model):
    def __init__(self, teacher1, teacher2, student, alpha=0.5, temperature=3.0):
        super(Distiller, self).__init__()
        self.teacher1 = teacher1
        self.teacher2 = teacher2
        self.student = student
        self.alpha = alpha
        self.temperature = temperature

        # Optimizer for the student
        self.student_optimizer = AdamW(learning_rate=0.0001)

    def compile(self, metrics, student_loss_fn, distillation_loss_fn):
        super(Distiller, self).compile(metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn

    def call(self, inputs):
        """
        This method defines how the model processes inputs.
        Args:
        - inputs: Input data (Xtest_student or any other data).

        Returns:
        - Output logits from the student model.
        """
        student_logits = self.student(inputs, training=False)
        return student_logits

    def train_step(self, data):
        x, y = data

        # Sequentially forward pass through teachers
        teacher1_logits = self.teacher1(x, training=False)
        teacher2_logits = self.teacher2(x, training=False)

        with tf.GradientTape() as tape:
            # Forward pass through student
            student_logits = self.student(x, training=True)

            # Compute the task loss (for hard labels)
            student_loss = self.student_loss_fn(y, student_logits)

            # Compute distillation losses (for soft labels)
            distill_loss1 = self.distillation_loss_fn(
                tf.nn.softmax(teacher1_logits / self.temperature, axis=1),
                tf.nn.softmax(student_logits / self.temperature, axis=1)
            ) * (self.temperature ** 2)

            distill_loss2 = self.distillation_loss_fn(
                tf.nn.softmax(teacher2_logits / self.temperature, axis=1),
                tf.nn.softmax(student_logits / self.temperature, axis=1)
            ) * (self.temperature ** 2)

            # Combined loss
            total_loss = self.alpha * student_loss + (1 - self.alpha) * (distill_loss1 + distill_loss2)

        # Compute gradients and update student model
        grads = tape.gradient(total_loss, self.student.trainable_variables)
        self.student_optimizer.apply_gradients(zip(grads, self.student.trainable_variables))

        # Update metrics for the student
        self.compiled_metrics.update_state(y, student_logits)

        # Return results
        results = {
            'student_loss': student_loss,
            'distill_loss1': distill_loss1,
            'distill_loss2': distill_loss2
        }
        results.update({m.name: m.result() for m in self.metrics})
        return results

    def test_step(self, data):
        x, y = data
        student_logits = self.student(x, training=False)

        # Compute the task loss (for hard labels)
        student_loss = self.student_loss_fn(y, student_logits)

        # Update metrics for the student
        self.compiled_metrics.update_state(y, student_logits)

        return {
            'student_loss': student_loss,
            **{m.name: m.result() for m in self.metrics}
        }



In [ ]:
# Transfer learning from Generic dl set to species
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold
from tensorflow.keras.optimizers import AdamW
from keras import models
k = 10
kf = KFold(n_splits=k, shuffle=True, random_state=42)
all_metrics = []
confusion_matrices = []

# K-fold cross-validation
for fold, (train_index, val_index) in enumerate(kf.split(Xtrain)):
    print(f"Training on fold {fold + 1}/{k}...")
    X_train_fold, X_val_fold = Xtrain[train_index], Xtrain[val_index]
    y_train_fold, y_val_fold = ytrain[train_index], ytrain[val_index]
    student_fold = create_student()
    teacher1_fold = models.load_model(path_model +"model_teacher21.h5") # Sử dụng teacher giống nhau trên cùng loài
    teacher2_fold = models.load_model(path_model +"model_teacher22.h5")
    distiller_fold = Distiller(teacher1=teacher1_fold, teacher2=teacher2_fold, student=student_fold, alpha=0.5, temperature=10)
    distiller_fold.compile(
        student_loss_fn=keras.losses.CategoricalCrossentropy(from_logits=True),
        metrics=["categorical_accuracy", "AUC"],
        distillation_loss_fn=keras.losses.KLDivergence(),
    )

    # Train model
    history = distiller_fold.fit(
        X_train_fold, y_train_fold,
        validation_data=(X_val_fold, y_val_fold),
        batch_size=16,
        epochs=50,

    )
    results = distiller_fold.evaluate(X_val_fold, y_val_fold)
    all_metrics.append(results)
    # confusion matrix and save TP, TN, FP, FN
    y_pred_fold = student_fold.predict(X_val_fold)
    y_pred_classes = np.argmax(y_pred_fold, axis=1)
    y_true_classes = np.argmax(y_val_fold, axis=1)

    # Tính confusion matrix và lưu TP, TN, FP, FN
    cm = confusion_matrix(y_true_classes, y_pred_classes)
    tn, fp, fn, tp = cm.ravel()
    confusion_matrices.append({"TP": tp, "TN": tn, "FP": fp, "FN": fn})
    print(f"Fold {fold + 1} - TP: {tp}, TN: {tn}, FP: {fp}, FN: {fn}")

# Average the results from the folds
avg_metrics = np.mean(all_metrics, axis=0)
print(f"\nAverage Metrics over {k}-Fold Cross Validation:")
for metric_name, avg_metric in zip(distiller_fold.metrics_names, avg_metrics):
    print(f"{metric_name}: {avg_metric:.4f}")

#  Show TP, TN, FP, FN
for fold, cm in enumerate(confusion_matrices):
    print(f"Fold {fold + 1} - TP: {cm['TP']}, TN: {cm['TN']}, FP: {cm['FP']}, FN: {cm['FN']}")
# save TP, TN, FP, FN into file Excel
df_confusion = pd.DataFrame(confusion_matrices)
df_confusion.to_excel(path_result + "cv_E.coli_KD2_Succi.xlsx", index=False)

print("\nConfusion matrix results saved to 'cv_E.coli_KD2_Succi.xlsx'")

Training on fold 1/10...


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/nn.py:567: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(
/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/trainer.py:582: UserWarning: `model.compiled_metrics()` is deprecated. Instead, use e.g.:
```
for metric in self.metrics:
    metric.update_state(y, y_pred)
```

  return self._compiled_metrics_update_state(


328/328 ━━━━━━━━━━━━━━━━━━━━ 28s 48ms/step - AUC: 0.5242 - categorical_accuracy: 0.5133 - distill_loss1: 0.0316 - distill_loss2: 0.0803 - loss: 0.5000 - student_loss: 0.6558 - val_AUC: 0.7175 - val_categorical_accuracy: 0.6278 - val_loss: 0.5000 - val_student_loss: 0.5302
Epoch 2/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 19s 45ms/step - AUC: 0.7276 - categorical_accuracy: 0.6719 - distill_loss1: 0.0258 - distill_loss2: 0.0698 - loss: 0.5000 - student_loss: 0.6040 - val_AUC: 0.7394 - val_categorical_accuracy: 0.6587 - val_loss: 0.5000 - val_student_loss: 0.5022
Epoch 3/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 21s 45ms/step - AUC: 0.7666 - categorical_accuracy: 0.6966 - distill_loss1: 0.0231 - distill_loss2: 0.0649 - loss: 0.5000 - student_loss: 0.5776 - val_AUC: 0.7694 - val_categorical_accuracy: 0.6930 - val_loss: 0.5000 - val_student_loss: 0.5629
Epoch 4/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 20s 43ms/step - AUC: 0.7851 - categorical_accuracy: 0.7058 - distill_loss1: 0.0206 - distill_loss2: 0.0605 - loss: 0.50

...

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Fold 9 - TP: 99, TN: 349, FP: 57, FN: 77
Training on fold 10/10...


Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/nn.py:567: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(
/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/trainer.py:582: UserWarning: `model.compiled_metrics()` is deprecated. Instead, use e.g.:
```
for metric in self.metrics:
    metric.update_state(y, y_pred)
```

  return self._compiled_metrics_update_state(


328/328 ━━━━━━━━━━━━━━━━━━━━ 27s 48ms/step - AUC: 0.6267 - categorical_accuracy: 0.5932 - distill_loss1: 0.0287 - distill_loss2: 0.0772 - loss: 0.5000 - student_loss: 0.6399 - val_AUC: 0.7435 - val_categorical_accuracy: 0.6942 - val_loss: 0.5000 - val_student_loss: 0.5391
Epoch 2/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 19s 43ms/step - AUC: 0.7366 - categorical_accuracy: 0.6750 - distill_loss1: 0.0253 - distill_loss2: 0.0695 - loss: 0.5000 - student_loss: 0.6013 - val_AUC: 0.7744 - val_categorical_accuracy: 0.7148 - val_loss: 0.5000 - val_student_loss: 0.4382
Epoch 3/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 15s 46ms/step - AUC: 0.7736 - categorical_accuracy: 0.7038 - distill_loss1: 0.0220 - distill_loss2: 0.0635 - loss: 0.5000 - student_loss: 0.5709 - val_AUC: 0.8030 - val_categorical_accuracy: 0.7268 - val_loss: 0.5000 - val_student_loss: 0.4114
Epoch 4/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 20s 45ms/step - AUC: 0.7857 - categorical_accuracy: 0.7117 - distill_loss1: 0.0196 - distill_loss2: 0.0592 - loss: 0.50

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold
from tensorflow.keras.optimizers import AdamW
k = 10
kf = KFold(n_splits=k, shuffle=True, random_state=42)
all_metrics = []
confusion_matrices = []

# K-fold cross-validation
for fold, (train_index, val_index) in enumerate(kf.split(Xtrain)):
    print(f"Training on fold {fold + 1}/{k}...")
    X_train_fold, X_val_fold = Xtrain[train_index], Xtrain[val_index]
    y_train_fold, y_val_fold = ytrain[train_index], ytrain[val_index]
    teacher1_fold = create_cnn_model()
    teacher2_fold = create_bilstm_model()
    student_fold = create_student()
    teacher1_fold.compile(
        loss=keras.losses.CategoricalCrossentropy(from_logits=True),
        metrics=[
            "categorical_accuracy",
            "AUC",
        ],
        optimizer=keras.optimizers.AdamW(learning_rate=0.0001),
    )
    teacher2_fold.compile(
        loss=keras.losses.CategoricalCrossentropy(from_logits=True),
        metrics=[
            "categorical_accuracy",
            "AUC",
        ],
        optimizer=keras.optimizers.AdamW(learning_rate=0.0001),
    )
    teacher1_fold.fit(
        X_train_fold, y_train_fold,
        batch_size =16,
        epochs=50
    )

    teacher2_fold.fit(
        X_train_fold, y_train_fold,
        batch_size =16,
        epochs=50
    )

    # Re-initialize the distiller for each fold
    distiller_fold = Distiller(teacher1=teacher1_fold, teacher2=teacher2_fold, student=student_fold, alpha=0.5, temperature=10)
    distiller_fold.compile(
        student_loss_fn=keras.losses.CategoricalCrossentropy(from_logits=True),

        metrics=["categorical_accuracy", "AUC"],
        distillation_loss_fn=keras.losses.KLDivergence(),

    )
    # Train model
    history = distiller_fold.fit(
        X_train_fold, y_train_fold,
        validation_data=(X_val_fold, y_val_fold),
        batch_size=16,
        epochs=50,
        verbose=1
    )

    # Save result of each fold
    results = distiller_fold.evaluate(X_val_fold, y_val_fold)
    all_metrics.append(results)
    y_pred_fold = student_fold.predict(X_val_fold)
    y_pred_classes = np.argmax(y_pred_fold, axis=1)
    y_true_classes = np.argmax(y_val_fold, axis=1)

     # confusion matrix và lưu TP, TN, FP, FN
    cm = confusion_matrix(y_true_classes, y_pred_classes)
    tn, fp, fn, tp = cm.ravel()
    confusion_matrices.append({"TP": tp, "TN": tn, "FP": fp, "FN": fn})
    print(f"Fold {fold + 1} - TP: {tp}, TN: {tn}, FP: {fp}, FN: {fn}")

# Average the results from the folds
avg_metrics = np.mean(all_metrics, axis=0)
print(f"\nAverage Metrics over {k}-Fold Cross Validation:")
for metric_name, avg_metric in zip(distiller_fold.metrics_names, avg_metrics):
    print(f"{metric_name}: {avg_metric:.4f}")

# Show TP, TN, FP, FN for each fold
for fold, cm in enumerate(confusion_matrices):
    print(f"Fold {fold + 1} - TP: {cm['TP']}, TN: {cm['TN']}, FP: {cm['FP']}, FN: {cm['FN']}")
# save TP, TN, FP, FN into the file Excel
df_confusion = pd.DataFrame(confusion_matrices)
df_confusion.to_excel(path_result + "cv_E.coli_KD2_Succ1.xlsx", index=False)

print("\nConfusion matrix results saved to 'cv_E.coli_KD2_Succ1.xlsx'")

Training on fold 1/10...


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/nn.py:567: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


328/328 ━━━━━━━━━━━━━━━━━━━━ 14s 22ms/step - AUC: 0.5626 - categorical_accuracy: 0.5427 - loss: 0.8596
Epoch 2/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 7s 20ms/step - AUC: 0.6602 - categorical_accuracy: 0.6350 - loss: 0.7044
Epoch 3/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 9s 27ms/step - AUC: 0.6648 - categorical_accuracy: 0.6288 - loss: 0.6755
Epoch 4/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 7s 19ms/step - AUC: 0.6653 - categorical_accuracy: 0.6261 - loss: 0.6659
Epoch 5/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 8s 13ms/step - AUC: 0.6602 - categorical_accuracy: 0.6371 - loss: 0.6717
Epoch 6/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - AUC: 0.6707 - categorical_accuracy: 0.6384 - loss: 0.6588
Epoch 7/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - AUC: 0.6890 - categorical_accuracy: 0.6557 - loss: 0.6450
Epoch 8/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - AUC: 0.6926 - categorical_accuracy: 0.6552 - loss: 0.6399
Epoch 9/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - AUC: 0.7075 - categorical_accuracy: 0.6621 - loss: 

...

328/328 ━━━━━━━━━━━━━━━━━━━━ 36s 55ms/step - AUC: 0.6578 - categorical_accuracy: 0.6262 - distill_loss1: 0.0221 - distill_loss2: 0.0978 - loss: 0.5000 - student_loss: 0.6369 - val_AUC: 0.7504 - val_categorical_accuracy: 0.6959 - val_loss: 0.5000 - val_student_loss: 0.5985
Epoch 2/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 19s 50ms/step - AUC: 0.7356 - categorical_accuracy: 0.6740 - distill_loss1: 0.0175 - distill_loss2: 0.0901 - loss: 0.5000 - student_loss: 0.6024 - val_AUC: 0.7685 - val_categorical_accuracy: 0.7062 - val_loss: 0.5000 - val_student_loss: 0.5945
Epoch 3/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 17s 51ms/step - AUC: 0.7710 - categorical_accuracy: 0.6927 - distill_loss1: 0.0138 - distill_loss2: 0.0838 - loss: 0.5000 - student_loss: 0.5717 - val_AUC: 0.7970 - val_categorical_accuracy: 0.7096 - val_loss: 0.5000 - val_student_loss: 0.5774
Epoch 4/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 17s 52ms/step - AUC: 0.7917 - categorical_accuracy: 0.7150 - distill_loss1: 0.0111 - distill_loss2: 0.0785 - loss: 0.50

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/nn.py:567: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


328/328 ━━━━━━━━━━━━━━━━━━━━ 10s 12ms/step - AUC: 0.5313 - categorical_accuracy: 0.5250 - loss: 0.9288
Epoch 2/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - AUC: 0.6284 - categorical_accuracy: 0.5971 - loss: 0.7344
Epoch 3/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 6s 18ms/step - AUC: 0.6412 - categorical_accuracy: 0.6135 - loss: 0.6957
Epoch 4/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step - AUC: 0.6590 - categorical_accuracy: 0.6273 - loss: 0.6750
Epoch 5/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - AUC: 0.6760 - categorical_accuracy: 0.6411 - loss: 0.6556
Epoch 6/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 6s 16ms/step - AUC: 0.6802 - categorical_accuracy: 0.6533 - loss: 0.6503
Epoch 7/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 9s 12ms/step - AUC: 0.6854 - categorical_accuracy: 0.6518 - loss: 0.6454
Epoch 8/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - AUC: 0.6827 - categorical_accuracy: 0.6603 - loss: 0.6461
Epoch 9/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 9s 12ms/step - AUC: 0.6642 - categorical_accuracy: 0.6352 - loss: 

/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/trainer.py:582: UserWarning: `model.compiled_metrics()` is deprecated. Instead, use e.g.:
```
for metric in self.metrics:
    metric.update_state(y, y_pred)
```

  return self._compiled_metrics_update_state(


328/328 ━━━━━━━━━━━━━━━━━━━━ 26s 50ms/step - AUC: 0.6689 - categorical_accuracy: 0.6300 - distill_loss1: 0.0150 - distill_loss2: 0.0967 - loss: 0.5000 - student_loss: 0.6352 - val_AUC: 0.7383 - val_categorical_accuracy: 0.6924 - val_loss: 0.5000 - val_student_loss: 0.5044
Epoch 2/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 15s 47ms/step - AUC: 0.7362 - categorical_accuracy: 0.6687 - distill_loss1: 0.0115 - distill_loss2: 0.0893 - loss: 0.5000 - student_loss: 0.5986 - val_AUC: 0.7735 - val_categorical_accuracy: 0.6976 - val_loss: 0.5000 - val_student_loss: 0.3776
Epoch 3/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 16s 48ms/step - AUC: 0.7829 - categorical_accuracy: 0.7108 - distill_loss1: 0.0092 - distill_loss2: 0.0826 - loss: 0.5000 - student_loss: 0.5647 - val_AUC: 0.8058 - val_categorical_accuracy: 0.7182 - val_loss: 0.5000 - val_student_loss: 0.3094
Epoch 4/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 20s 47ms/step - AUC: 0.7857 - categorical_accuracy: 0.7018 - distill_loss1: 0.0087 - distill_loss2: 0.0784 - loss: 0.50